# ２つの平均値を比較する
この章では、２つの平均値を比較するための方法を学ぶ

## ２つの平均値を比較するケース
２つの平均値を比較することが必要となるのはどんな場合か。
- 男女で心理学テストの平均値に差があるかを検討したい
- 統計が好き・嫌いで統計テストの平均値に差があるかを検定したい
- 統計の指導を受けたことにより成績が向上するかを検討したい

<p>などがあるが、こうした疑問に答えるために検定を行う。独立な2群のt検定を取り上げる。上記の例では最初の２つがこの方法で分析できるケースになる。最後の一つは対応のあるt検定である</p>

## 独立な2群のt検定
まず独立な2群のt検定のための検定統計量から考えていく。平均誤差の標本分布は
$$
\overline{X_1} - \overline{X_2}〜N(\mu_1 - \mu_2, \sigma^2(\frac{1}{n_1} + \frac{1}{n_2}))
$$
となる。これを標準化すると以下のようになる
$$
\frac{\overline{X_1} - \overline{X_2} - (\mu_1 - \mu_2)}{\sigma\sqrt{(\frac{1}{n_1} + \frac{1}{n_2})}}〜N(0, 1)
$$

標準化を行うことにより、標準正規分布N(0,1)に従うようになった。検定統計量の分母に未知の値である母標準偏差$\sigma$が含まれているが、この値は直接求められないため、次式で推定する。
$$
\hat{\sigma}_{pooled}^2=\frac{(n_1 - 1)\hat{\sigma_1}^2+(n_2 - 1)\hat{\sigma_2}^2}{n_1 + n_2 - 2}
$$
この式は、$\hat{\sigma_1}^2$と$\hat{\sigma_2}^2$はそれぞれの群の不偏分散を$n_1$と$n_2$はそれぞれの群のサンプルサイズを示している。この母分散の推定量$\hat{\sigma}_{pooled}$のことを、２群をプールした分散という。これは2群に共通の母分散$\sigma^2$の不偏推定量である。検定統計量の分母にある未知の$\sigma$を$\hat{\sigma}_{pooled}$によって推定したものが下記の検定統計量となる。
$$
t = \frac{\overline{X_1} - \overline{X_2} - (\mu_1 - \mu_2)}{\sqrt{\frac{(n_1 - 1)\hat{\sigma_1}^2+(n_2 - 1)\hat{\sigma_2}^2}{n_1 + n_2 - 2}(\frac{1}{n_1} + \frac{1}{n_2})}}$$
となるが、ここで帰無仮説は$\mu_1 = \mu_2$となるため、$\mu_1 - \mu_2=0$となるため最終的は下記の式となる。
$$
t = \frac{\overline{X_1} - \overline{X_2}}{\sqrt{\frac{(n_1 - 1)\hat{\sigma_1}^2+(n_2 - 1)\hat{\sigma_2}^2}{n_1 + n_2 - 2}(\frac{1}{n_1} + \frac{1}{n_2})}}$$
となる。こうして導かれた検定統計量tの標本分布は帰無仮説$\mu_1=\mu_2$のもとで、自由度$df=n_1 + n_2 - 2$のt分布に従うことが知られている。この検定統計量を用いて、２つの平均値の差に関する検定を行うことができる。

<b>
例題：「統計テスト1」の得点の平均値に男女で有意な差があるか有意水準5%両側検定で検定しよう
</b>

In [19]:
import pandas as pd
import numpy as np
from scipy import stats
data = pd.read_csv('../data/teaching_method.csv')

In [3]:
data.head()

,id,name,sex,math,stats,psychology_test,stats_test1,stats_test2,teaching_method
0,1,大村,男,嫌い,好き,13,6,10,C
1,2,本多,男,嫌い,好き,14,10,13,B
2,3,川崎,男,好き,好き,7,6,8,B
3,4,多村,男,好き,好き,12,10,15,A
4,5,松中,男,嫌い,嫌い,10,5,8,B


In [4]:
mens = data[data.sex == '男']['stats_test1']
womens = data[data.sex == '女']['stats_test1']

### (1)帰無仮説と対立仮説の設定
- 帰無仮説$H_0$：$\mu_1 = \mu_2$（２つの母平均は等しい）
- 帰無仮説$H_1$：$\mu_1 \neq \mu_2$（２つの母平均は等しくない）
<p>対立仮説よりこの検定は両側検定で行う</p>

### (2)検定統計量の選択
$$
t = \frac{\overline{X_1} - \overline{X_2}}{\sqrt{\frac{(n_1 - 1)\hat{\sigma_1}^2+(n_2 - 1)\hat{\sigma_2}^2}{n_1 + n_2 - 2}(\frac{1}{n_1} + \frac{1}{n_2})}}$$
を検定統計量をする。そして、この検定統計量は帰無仮説のもとで自由度$df=n_1+n_2-2$のt分布に従う。

### (3)有意水準$\alpha$の決定
有意水準は5%、つまり$\alpha=0.05$とする、両側検定である。

### (4)検定統計量の実現値を求める
まずは平均と不偏分散を求めてみよう。

In [7]:
print(mens.mean())
print(womens.mean())
print(mens.var())
print(womens.var())

6.0
8.0
7.777777777777778
4.0


$$
\hat{\sigma}_{pooled}=\sqrt{\frac{(n_1 - 1)\hat{\sigma_1}^2+(n_2 - 1)\hat{\sigma_2}^2}{n_1 + n_2 - 2}}
$$
こちらを求める。

In [11]:
sigma_bool = np.sqrt(
( (len(mens) - 1)* mens.var() + (len(womens) - 1)* womens.var() ) / ( len(mens) + len(womens) - 2 )
)
sigma_bool

2.4267032964268394

検定統計量tの分母を求める

In [14]:
t_denominator = sigma_bool * np.sqrt( ( 1/len(mens) ) + ( 1/len(womens) ) )
t_denominator

1.0852547064066471

In [16]:
t_numerator = mens.mean() - womens.mean()
t_numerator

-2.0

In [18]:
t = t_numerator / t_denominator
t

-1.8428853505018534

検定統計量の実現値が$t=-1.842885$と求まった。

### (5)帰無仮説の棄却 or 採択の決定

In [24]:
df = len(mens) + len(womens) - 2
under_p = stats.t.ppf(0.025,df)     # 自由度18のt分布で下側確率0.025となるtの値を求める
upper_p = stats.t.ppf(1 - 0.025,df) # 自由度18のt分布で上側確率0.025となるtの値を求める

In [26]:
# 棄却域に入るかどうかのチェック
t < under_p or t > upper_p

False

棄却域に入らないので、帰無仮説は棄却されない（5%の水準で有意差は見られなかった）。つまり男女で有意な差はないとなる。

## t検定の前提条件
下記の項目を満たしている必要がある
- 標本抽出が無作為に行われていること
- 母集団分布が正規分布にしたがっていること
- ２つの母集団の分散が等質であること

### 分散の等質性の検定（F検定）
<p>分散の等質性の検定を行う(２つの群の分散が等しいこと)。この検定では、分散は等質であるという帰無仮説が採択されることにより、２つの母分散は等質である。つまり分散の等質性の過程が満たされていると判断することになり、t検定を行うことができる。</p>
<p>
これらを検定する方法をF検定と呼ぶ。検定統計量Fを求める式は以下である。
    $$
    F = \frac{\hat{\sigma_1}^2}{\hat{\sigma_2}^2} (\hat{\sigma_1}^2 < \hat{\sigma_2}^2)
     $$
    各群の不偏分散を求め、値が大きい方が分母に来るようにして、F値を求める。F分布は各群の自由度が必要となる。各群のデータ数を$n_1、n_2$とすると自由度${df}_1、{df}_2$は
    $$
    {df}_1 = n_1 - 1 \\
    {df}_2 = n_2 - 1
    $$
    あとはこれらの値をF分布を使って棄却域に入るかどうかを調べる形となる。
</p>
<b>例題：統計学の授業にクラスAとクラスBという２つのクラスがあり、テストを実施した。クラスAとクラスBの平均点に有意な差があると言えるか。有意水準5%で答えよ</b>

In [28]:
class_a = np.array([54, 55, 52, 48, 50, 38, 41, 40, 53, 52])
class_b = np.array([67, 63, 50, 60, 61, 69, 43, 58, 36, 29])
print(class_a.var(), class_b.var())

35.809999999999995 166.04000000000002


In [30]:
f = class_a.var() / class_b.var()
stats.f.cdf(f, len(class_a)-1, len(class_b)-1)*2

0.032059108242914004

In [32]:
df1 = len(class_a) - 1
df2 = len(class_b) - 1

under_p = stats.f.ppf(0.025, df1, df2)     # F分布で下側確率0.025となるtの値を求める
upper_p = stats.f.ppf(1 - 0.025, df1, df2) # F分布で上側確率0.025となるtの値を求める

In [36]:
f < under_p or f > upper_p

True

ここで帰無仮説（２つの群は等質である）は棄却されるため、対立仮説（２つの群は等質ではない）が採用される。そのため、t検定の条件を満たさない。t検定の条件を満たさない場合でも利用できるWelch(ウェルチ)の検定を紹介する